<a href="https://colab.research.google.com/github/fmipython/PythonCourse2022/blob/17_c_code_in_python/17%20-%20Using%20C%20code%20in%20Python/17%20-%20Using%20C%20code%20in%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Използване на C код в Python

План на лекцията:
- [ ] Защо C код в Python ?
- [ ] Как работят C библиотеките ?
- [X] Простичка функция
- [ ] Структури
- [ ] Скорост
- [ ] Примери
- [ ] Задачи

## Предварителна подготовка

In [ ]:
!git clone https://github.com/fmipython/PythonCourse2022.git && cd PythonCourse2022

fatal: destination path 'PythonCourse2022' already exists and is not an empty directory.
Already on '17_c_code_in_python'
Your branch is up to date with 'origin/17_c_code_in_python'.
Already up to date.


## Защо C код в Python ?

## Как работят C библиотеките ?

## Съпоставяне на типовете от C в Python

Основната библиотека, чрез която ще реализираме връзката между C код и Python, е `ctypes`. В нея е всичко необходимо за използването на външен C код в Python.

Както знаем, типовете в Python не са същите както в C. Python дефинира типове, имащи за цел да представят типовете в C. Те са разделени на три категории:

1. Прости (fundamental) типове
2. Сложни (structural) типове
3. Масиви и указатели

### Прости типове

Простите типове са:
- `c_char` - съотвества на C типа `char`
- `c_char_p` - съотвества на C типа `char*`
- `c_double` - съотвества на C типа `double`
- `c_float` - съотвества на C типа `float`
- `c_int` - съотвества на C типа `int`
- `c_longlong` - съотвества на C типа `long long`
- `c_short` - съотвества на C типа `short`
- `c_size_t`- съотвества на C типа `size_t`
- `c_uint` - съотвества на C типа `unsigned int`
- `c_void_p` - съотвества на C типа `void*`
- `c_bool` - съотвества на C типа `bool`
- и други... (може да откриете пълния списък [тук](https://docs.python.org/3.10/library/ctypes.html?highlight=cdll#ctypes-fundamental-data-types-2))

Когато една C функция върне прост тип, този тип автоматично се конвертира в подходящ Python тип.

## Простичка функция

За да демонстрираме практически как можем да използваме C код в Python, ще съзадем C функция, която ще има за цел да събере две числа.

За целта ще използваме вече написан C код, както и предварително подготвен `CMakeLists.txt` файл. За да изпълните успешно кода, ще ви е нужен инсталиран `CMake`, както и C компилатор. 

Може да разгледате C кода [тук](https://github.com/fmipython/PythonCourse2022/tree/main/17%20-%20Using%20C%20code%20in%20Python/C/example_1).

In [1]:
!cat "C/example_1/sum.h"

#ifndef EXAMPLE_1_SUM_H
#define EXAMPLE_1_SUM_H

extern int sum(const int a, const int b);

#endif

In [3]:
!cat "C/example_1/sum.c"

#include "sum.h"

int sum(const int a, const int b) {
    return a + b;
}

Единствената съществена разлика на този етап в C кода, е използването на ключовата дума `extern` в началото на декларацията на функцията. `extern` променя видимостта на функция, така че да е видима във външни библиотеки.

Компилираме нашия код до C библиотека с помощта на `cmake` и `make` командите

In [7]:
!cd "C/example_1" && cmake . && make

-- The C compiler identification is GNU 12.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/lyubolp/PythonCourse2022/17 - Using C code in Python/C/example_1
[ 50%] Building C object CMakeFiles/Example1.dir/sum.c.o
[100%] Linking C shared library libExample1.so
[100%] Built target Example1


След като вече имаме `libExample1.so` файла, можем да пристъпим към зареждането ѝ в Python.

Основната библиотека, която ще използваме е `ctypes`. Можем да заредим външна C библиотека с помощта на `ctypes.CDLL` функцията. Тя връща специален обект `CDLL` обект, който съдържа заредената бибилиотека.

След като успешно заредим нашата библиотека, в новополучената ни инстанция ще се появят атрибути, които са класове от тип `_FuncPtr` - те ще сочат към функциите в нашата C библиотека.
По поздразибране те приемат всякакви `ctypes` аргументи и връщат резултат по подразбиране. Можем да специфицираме аргументите и типа на резултата чрез атрибутите `argtypes` и `restype`.

In [14]:
import ctypes
import os

lib_path = os.path.join("C", "example_1", "libExample1.so")

def setup_lib(path: str) -> ctypes.CDLL:
    lib = ctypes.CDLL(path)
    print(type(lib.sum))
    lib.sum.argtypes = [ctypes.c_int, ctypes.c_int]
    lib.sum.restype = ctypes.c_int
    return lib

lib = setup_lib(lib_path)
print(type(lib))

result = lib.sum(2, 3)
print(type(result), result)

<class 'ctypes.CDLL.__init__.<locals>._FuncPtr'>
<class 'ctypes.CDLL'>
<class 'int'> 5


In [11]:
import ctypes
import os

lib_path = os.path.join("C", "example_1", "libExample1.so")

def setup_lib(path: str) -> ctypes.CDLL:
    lib = ctypes.CDLL(path)
    print(type(lib.sum))
    lib.sum.argtypes = [ctypes.c_int, ctypes.c_int]
    lib.sum.restype = ctypes.c_int
    return lib

lib = setup_lib(lib_path)
print(type(lib))
a = int(input("Enter first number: "))
b = int(input("Enter second number: "))

result = lib.sum(a, b)

print(type(result))
print("{} + {} = {}".format(a, b, result))


<class 'ctypes.CDLL.__init__.<locals>._FuncPtr'>
<class 'ctypes.CDLL'>


ValueError: invalid literal for int() with base 10: ''

Нека разгледаме в детайли кода.

Използваме `ctypes` библиотеката за работа с външни C библиотеки. За зареждането на библиотеката използваме `CDLL` конструктора, като му подаваме пътя към библиотеката. 

След това е необходимо да посочим типа на аргументите и типа на резултата. Понеже работим с `int` променливи, типа на аргументите и резултата са `c_int`. 

С получения обект, можем да извикваме функциите, които са отбелязани като `extern` в C кода.

## Структури

Нека усложним нещата една идея - нека се опитаме да подаваме C структури към нашия Python code.

Ще дефинираме структурата `Rational`, която ще моделира рационално число, съставено от две цели числа - числител и знаменател. Ще дефинираме също функции за събиране, изваждане, умножение, деление, както и функция, която конструира нов `Rational` обект на базата на две цели числа.

Целия C код може да разгледате [тук](https://github.com/fmipython/PythonCourse2022/tree/main/17%20-%20Using%20C%20code%20in%20Python/C/example_2).

In [2]:
!cd "C/example_2" && cmake . && make

-- The C compiler identification is GNU 12.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/lyubolp/PythonCourse2022/17 - Using C code in Python/C/example_2
[ 50%] Building C object CMakeFiles/Example2.dir/rational.c.o
[100%] Linking C shared library libExample2.so
[100%] Built target Example2


Разликата с предишния пример е, че този път имаме C структура.
Как бихме могли да представим нашата структура `Rational` в Python ? Класът `ctypes.Structure` ни служи като база, върху която да създадем нашия `Rational` клас в Python. Чрез специалната клас-променлива `_fields_` можем да зададем от какви променливи е създадена нашата структура - в случая на `Rational`, две променливи от тип `int`.

In [ ]:
import ctypes

class Rational(ctypes.Structure):
    _fields_ = [("numerator", ctypes.c_int), ("denominator", ctypes.c_int)]

    def __str__(self):
        return str(self.numerator) + "/" + str(self.denominator)


Понеже нашите C структури са представени като Python класове, ние може да дефинираме допълнителни Python методи в тях - в примера сме дефинирали метода `__str__` - него можем да използваме когато работим с нашата C структура през Python.

Зареждането на останалите функции става по познатия ни начин.

In [4]:
import ctypes
import os


def setup_lib(path) -> ctypes.CDLL:
    lib = ctypes.CDLL(path)

    lib.add.argtypes = [Rational, Rational]
    lib.add.restype = Rational

    lib.subtract.argtypes = [Rational, Rational]
    lib.subtract.restype = Rational

    lib.multiply.argtypes = [Rational, Rational]
    lib.multiply.restype = Rational

    lib.divide.argtypes = [Rational, Rational]
    lib.divide.restype = Rational

    lib.build.argtypes = [ctypes.c_int, ctypes.c_int]
    lib.build.restype = Rational

    return lib

lib_path = os.path.join("C", "example_2", "libExample2.so")

lib = setup_lib(lib_path)

first_num = int(input("Enter first number numerator: "))
first_denom = int(input("Enter first number denominator: "))

first_rational = lib.build(first_num, first_denom)


second_num = int(input("Enter second number numerator: "))
second_denom = int(input("Enter second number denominator: "))
second_rational = lib.build(second_num, second_denom)

add = lib.add(first_rational, second_rational)
subtract = lib.subtract(first_rational, second_rational)
multiply = lib.multiply(first_rational, second_rational)
divide = lib.divide(first_rational, second_rational)

print(f"{first_rational} + {second_rational} = {add}")
print(f"{first_rational} - {second_rational} = {subtract}")
print(f"{first_rational} * {second_rational} = {multiply}")
print(f"{first_rational} / {second_rational} = {divide}")


2/3 + 4/5 = 22/15
2/3 - 4/5 = -2/15
2/3 * 4/5 = 8/15
2/3 / 4/5 = 10/12


## C++ код

## Скорост

## Примери

## Задачи